# Setup

In [10]:
import sys
import importlib

src_path = '/Users/jblandin/my/school-local/cs511/pywumpuslite'

def reload_module_by_name(mod_name, var_name, path):
    if path is not None and path not in sys.path:
        sys.path.append(path)
        print('Appended {} to sys.path'.format(path))
    else:
        print('{} already in sys.path'.format(path))
                
    for mod in list(sys.modules.keys()):
        if mod_name in mod:
            del sys.modules[mod]
    
    if var_name in globals():
        del globals()[var_name] # deletes the variable named <var_name>
        
    return importlib.__import__(mod_name)

def reload(path=src_path):
    global world_application
    world_application = reload_module_by_name('world_application', 'world_application', path=path)
    print('\world_application.__file__: {}'.format(world_application.__file__))
    import world_application
    
reload(src_path)

/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
\world_application.__file__: /Users/jblandin/my/school-local/cs511/pywumpuslite/world_application.py


# Run

In [11]:
reload()
print()
out_writer = open('wumpus_output.txt', 'w')

world_application.run_application(num_trials=2)

/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
\world_application.__file__: /Users/jblandin/my/school-local/cs511/pywumpuslite/world_application.py

Dimensions: 4 x 4
Maximum number of steps: 50
Number of trials: 2
Random agent location: False
Random number seed: 361929
Output filename: wumpus_output.txt
Non-Deterministic Behavior: True

 -----------------------
|     |     | P W |     | 
|     |     |     |     | 
 -----------------------
|     |     |     |     | 
|     |     |     |     | 
 -----------------------
|     |     |     | P   | 
|     | G   |     |     | 
 -----------------------
|     |     |     |     | 
|   > |     |     |     | 
 -----------------------


Percept: < none, none, none, none, none >

Current score: 0
Last action: START_TRIAL
Time step: 0

 -----------------------
|     |     | P W |     | 
|     |     |     |     | 
 -----------------------
|     |     |     |     | 
|     |     |     |     | 
 -----------------------
|     |    